# GomMold Mold Detection – YOLOv8 Training Notebook

**Course**: Software Engineering and AI Application

**Team Members**
- Nur Shaqeerah
- Aina Nur Insyeerah
- Nur Ain Syafiqah
- Yumni Karmila

**Purpose**:  
This notebook trains and evaluates a YOLOv8 model for detecting mold and non-mold areas in household environments.


In [ ]:
#install prerequisites
!pip install ultralytics


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## 1. Dataset

The dataset was created using Roboflow (mold_detection_GomMold).  
It contains two classes:

- **Mold**
- **No Mold**

The dataset has been augmented with:
- Horizontal flips  
- Brightness/contrast adjustments  
- Exposure variations  

### 🔗 Roboflow Dataset Link
Dataset source (Roboflow project page):

👉 https://universe.roboflow.com/gommold-8opye/mold_detection_gommold-if0p0

This link provides access to dataset details, versions, and annotations.
(download the latest version from there)


In [ ]:

# Download Dataset from Roboflow
!pip install roboflow
from roboflow import Roboflow

# ⚠️ IMPORTANT:
# Insert your own Roboflow API key here if you want to reproduce the experiment.
# The key has been removed for security in the public GitHub version.

rf = Roboflow(api_key="API-KEY")
project = rf.workspace("gommold-8opye").project("mold_detection_gommold-if0p0")
version = project.version(4)
dataset = version.download("yolov8")




In [ ]:
#if u downloaded the data manually from the link, unzip the file

!unzip /content/mold_detection_GomMold.yolov8.zip -d /content/


## 2. Load YOLOv8 Model
We start training from `yolov8n.pt` using transfer learning.

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

## 3. Train the Model


In [ ]:
model.train(
    data="/content/mold_detection_GomMold-4/data.yaml",  # path to dataset
    epochs=50,        # 30–50 recommended
    imgsz=640,        # image size
    batch=16,
    patience=10,      # early stopping if no improvement
    lr0=0.01,         # initial learning rate
    augment=True      # enable augmentation
)

In [ ]:
# Mount the trained model to drive

!cp /content/runs/detect/train/weights/best.pt /content/gdrive/MyDrive/GomMold_AIML/new_best4.pt #change this to your drive directory


## 4. Training Metrics & Graphs
YOLO automatically generates:
- results.png
- loss curves
- mAP curves
- confusion matrix


In [ ]:
from IPython.display import Image

Image('/content/runs/detect/train/results.png')


In [ ]:
Image('/content/runs/detect/train/confusion_matrix.png')

In [ ]:
!pip install firebase-admin


In [ ]:
#export as ONNX for backend

from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/best.pt")

model.export(format="onnx", opset=12)


## 5. Inference Samples


In [ ]:
!mkdir sample_test

# copy a few images from test set
!cp /content/mold_detection_GomMold-4/test/images/*.jpg sample_test/

# run prediction
results = model.predict("/content/sample_test", save=True)


## 8. Conclusion

The YOLOv8 model performed well in detecting mold vs non-mold images.  
Further improvements include:
- Increasing dataset variety
- Improving low-light images
- Adding more “No Mold” clean wall images to reduce false positives
